In [1]:
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies')

# import alchemy_tools_new as atn
import alchemy_tools as at
import alchemy_tools2 as atn
import glob
import numpy as np
import scipy
from parse_density_files import CUBE

import matplotlib
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 24})

def get_paths(directory):
    # load data from cube files
    # paths_cubes = ['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/ueg/ve_00.cube'] # at avl24
    paths_cubes = ['/home/misa/projects/Atomic-Energies/data/ueg_reference/ueg/box20/ve_00.cube'] # at avl51
    paths2 = glob.glob(directory+'/cube-files/ve*')
    paths2.sort()
    paths_cubes.extend(paths2)
    return(paths_cubes)

# Influence of number of integration points along $\lambda$

### using all lambda values

In [2]:
#paths_cubes = get_paths('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003886')
paths_cubes = get_paths('/home/misa/projects/Atomic-Energies/data/dsgdb9nsd_003664')
lam_vals, densities, nuclei, gpts, h_matrix = atn.load_cube_data(paths_cubes)

In [5]:
#results_dsgdb9nsd_003886 = atn.atomic_energy_decomposition(lam_vals, densities, nuclei, gpts, h_matrix, intgr_method='trapz')
results_dsgdb9nsd_003664 = atn.atomic_energy_decomposition(lam_vals, densities, nuclei, gpts, h_matrix, intgr_method='trapz')

In [ ]:
paths_cubes = get_paths('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212')
paths_cubes = paths_cubes[:-1]
lam_vals, densities, nuclei, gpts, h_matrix = atn.load_cube_data(paths_cubes)

In [ ]:
results_dsgdb9nsd_001212 = atn.atomic_energy_decomposition(lam_vals, densities, nuclei, gpts, h_matrix, intgr_method='trapz')

### using 6 values

In [ ]:
paths_cubes = get_paths('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003886')
small_cube_set = [0,3,6,11,15,19]
small_cube_set_dsgdb9nsd_003886 = []
for i in small_cube_set:
    small_cube_set_dsgdb9nsd_003886.append(paths_cubes[i])
lam_vals, densities, nuclei, gpts, h_matrix = atn.load_cube_data(small_cube_set_dsgdb9nsd_003886)
results_dsgdb9nsd_003886_small_set = atn.atomic_energy_decomposition(lam_vals, densities, nuclei, gpts, h_matrix, intgr_method='trapz')

In [ ]:
paths_cubes = get_paths('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212')
small_cube_set = [0,3,7,12,17,21]
small_cube_set_dsgdb9nsd_001212 = []
for i in small_cube_set:
    small_cube_set_dsgdb9nsd_001212.append(paths_cubes[i])
lam_vals, densities, nuclei, gpts, h_matrix = atn.load_cube_data(small_cube_set_dsgdb9nsd_001212)
results_dsgdb9nsd_001212_small_set = atn.atomic_energy_decomposition(lam_vals, densities, nuclei, gpts, h_matrix, intgr_method='trapz')

#### Difference between same atoms

In [ ]:
diff_same_atom = np.concatenate((results_dsgdb9nsd_003886[2] - results_dsgdb9nsd_003886_small_set[2], results_dsgdb9nsd_001212[2] - results_dsgdb9nsd_001212_small_set[2]))

In [ ]:
diff_same_atom.mean()

In [ ]:
diff_same_atom.std()

In [ ]:
histogram = np.histogram(diff_same_atom)
plt.hist(diff_same_atom, bins = 20)

In [ ]:
diff = []
for i in results_dsgdb9nsd_001212[2]:
    diff.append(results_dsgdb9nsd_001212[2][0] - i)
    
for j in results_dsgdb9nsd_003886[2]:
    diff.append(results_dsgdb9nsd_001212[2][0] - j)

In [ ]:
diff

In [ ]:
diff_small = []
for i in results_dsgdb9nsd_001212_small_set[2]:
    diff_small.append(results_dsgdb9nsd_001212_small_set[2][0] - i)
    
for j in results_dsgdb9nsd_003886_small_set[2]:
    diff_small.append(results_dsgdb9nsd_001212_small_set[2][0] - j)

In [ ]:
diff_small

In [ ]:
np.array(diff)-np.array(diff_small)

# Some old stuff

In [ ]:
# paths_cubes = get_paths('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212')
paths_cubes = get_paths('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003886')
paths_cubes

In [ ]:
# load the cube-files

compound = 'dsgdb9nsd_001212'
paths_cubes = get_paths('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212')

# compound = 'dsgdb9nsd_003712'
# paths_cubes = get_paths('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003712')

# compound = 'dsgdb9nsd_003886'
# paths_cubes = get_paths('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003886')



lam_vals, densities, nuclei, gpts, h_matrix = atn.load_cube_data(paths_cubes)

In [ ]:
nuclei

In [ ]:
def atomic_energies_piecewise_integration(intervals_idc, method, lam_vals, densities, nuclei, gpts, h_matrix):
    """
    integrate over lambda and r for subintervals of lambda
    intervals_idc: indices of the intervals over which to integrate lambda; 
    e.g. lam_vals[intervals_idc[0]] would give the first interval for integraion
    method: integration method (trapz or cubic-splines)
    lam_vals: lambda values for which the density was calculated
    densities: the densities corresponding to the lambda-values in lam_vals
    nuclei: the positions of the nuclei
    gpts: the coordinates of the grid on which the density is represented
    h_matrix: necessary to calculate distance between gridpoints and nuclei with minimum image convention
       """
    # integrate for all intervals
    atomic_energies_new = []
    atomic_energies = []

    if method == 'trapz':

        for idc in intervals_idc:
            for i in range(len(idc)-1):
                # integration over lambda from lower limit to upper limit
                lower_limit = idc[i]
                upper_limit = idc[i+1]
                int_limits = [lower_limit, upper_limit]
                av_dens = np.trapz(densities[int_limits], x=lam_vals[int_limits], axis=0)
                atomic_energies_new_interval, alch_pots_new_interval = atn.calculate_atomic_energies(av_dens, nuclei, gpts, h_matrix)
                atomic_energies_interval, alch_pots_interval = at.calculate_atomic_energies(av_dens, nuclei, gpts)

                # store energy per interval
                atomic_energies_new.append(atomic_energies_new_interval)
                atomic_energies.append(atomic_energies_interval)

    if method == 'cubic-splines':

        # get all lambda values for generation of splines
        unique_idc = []
        for el in intervals_idc:
            unique_idc.extend(el.tolist())
        # list(set(unique_idc))
        unique_idc = list(set(unique_idc))
        unique_idc.sort()

        # create splines
        poly_obj = scipy.interpolate.CubicSpline(lam_vals[unique_idc], densities[unique_idc], axis=0, bc_type=('clamped', 'not-a-knot'))

        # integrate over the intervals
        for idc in intervals_idc:
            for i in range(len(idc)-1):
                # integration over lambda from lower limit to upper limit
                lower_limit = idc[i]
                upper_limit = idc[i+1]

                av_dens = poly_obj.integrate(lam_vals[lower_limit], lam_vals[upper_limit])
                atomic_energies_new_interval, alch_pots_new_interval = atn.calculate_atomic_energies(av_dens, nuclei, gpts, h_matrix)
                atomic_energies_interval, alch_pots_interval = at.calculate_atomic_energies(av_dens, nuclei, gpts)

                # store energy per interval
                atomic_energies_new.append(atomic_energies_new_interval)
                atomic_energies.append(atomic_energies_interval)

    atomic_energies_new = np.array(atomic_energies_new)
    atomic_energies = np.array(atomic_energies)
    
    return(atomic_energies_new, atomic_energies)

## Atomic Energies with 6 $\lambda$-values

In [ ]:
# define intervals, lambda-values
I1 = [0, 8/38]
I2 = [8/38, 15/38]
I3 = [15/38, 23/38]
I4 = [23/38, 30/38]
I5 = [30/38, 38/38]

intervals_small = [I1, I2, I3, I4, I5]
intervals_idc_small = []

for i in intervals_small:
    intervals_idc_small.extend(np.where(np.isin(lam_vals, i)))

In [ ]:
intervals_idc_small

In [ ]:
# integration with trapezoidal rule
ae_small_MIC_trapz, ae_small_trapz = atomic_energies_piecewise_integration(intervals_idc_small, 'trapz', lam_vals, densities, nuclei, gpts, h_matrix)

In [ ]:
# integration with cubic splines
ae_small_MIC_cs, ae_small_cs = atomic_energies_piecewise_integration(intervals_idc_small, 'cubic-splines', lam_vals, densities, nuclei, gpts, h_matrix)

In [ ]:
# trapezoidal
header_small_trapz = 'atomic energies for dsgdb9nsd_001212 for piecewise integration over lambda with the trapezoidal rule\n'
header_small_trapz += 'every line corresponds to an integration interval, the intervals are [0, 8]/38, [8,15]/38, [15, 23]/38, [23,30]/38, [30, 38]/38\n'
header_small_trapz += 'every column corresponds to an atom where the sum of values in a column is the atomic energy of the atom'
path = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/integration/'
path_ae_small_trapz = path + 'ae_small_trapz.txt'
np.savetxt(path_ae_small_trapz, ae_small_trapz, delimiter='\t', header=header_small_trapz)
# trapezoidal mic
header_small_trapz_mic = 'atomic energies for dsgdb9nsd_001212 for piecewise integration over lambda with the trapezoidal rule and the minimum image convention for integration over r\n'
header_small_trapz_mic += 'every line corresponds to an integration interval, the intervals are [0, 8]/38, [8,15]/38, [15, 23]/38, [23,30]/38, [30, 38]/38\n'
header_small_trapz_mic += 'every column corresponds to an atom where the sum of values in a column is the atomic energy of the atom'
path = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/integration/'
path_ae_small_trapz_mic = path + 'ae_small_MIC_trapz.txt'
np.savetxt(path_ae_small_trapz_mic, ae_small_MIC_trapz, delimiter='\t', header=header_small_trapz_mic)

# cubic splines
header_small_cs = 'atomic energies for dsgdb9nsd_001212 for piecewise integration over lambda using cubic splines\n'
header_small_cs += 'every line corresponds to an integration interval, the intervals are [0, 8]/38, [8,15]/38, [15, 23]/38, [23,30]/38, [30, 38]/38\n'
header_small_cs += 'every column corresponds to an atom where the sum of values in a column is the atomic energy of the atom'
path = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/integration/'
path_ae_small_cs = path + 'ae_small_cs.txt'
np.savetxt(path_ae_small_cs, ae_small_cs, delimiter='\t', header=header_small_cs)
# cubic splines mic
header_small_cs_mic = 'atomic energies for dsgdb9nsd_001212 for piecewise integration over lambda using cubic splines and the minimum image convention for integration over r\n'
header_small_cs_mic += 'every line corresponds to an integration interval, the intervals are [0, 8]/38, [8,15]/38, [15, 23]/38, [23,30]/38, [30, 38]/38\n'
header_small_cs_mic += 'every column corresponds to an atom where the sum of values in a column is the atomic energy of the atom'
path = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/integration/'
path_ae_small_cs_mic = path + 'ae_small_MIC_cs.txt'
np.savetxt(path_ae_small_cs_mic, ae_small_MIC_cs, delimiter='\t', header=header_small_cs_mic)

## Atomic Energies with all $\lambda$-values

In [ ]:
# dsgdb9nsd_001212
I1_large = np.array([0, 4, 6, 8])/38
I2_large = np.array([8, 10, 12, 14, 15])/38
I3_large = np.array([15, 16, 18, 20, 22, 23])/38
I4_large = np.array([23, 24, 26, 28, 29, 30])/38
I5_large = np.array([30, 32, 34, 36, 38])/38

# dsgdb9nsd_003712
# I1_large = np.array([0, 4, 6, 8])/38
# I2_large = np.array([8, 10, 12, 14, 15])/38
# I3_large = np.array([15, 16, 18, 20, 22, 23])/38
# I4_large = np.array([23, 24, 26, 28, 30])/38
# I5_large = np.array([30, 32, 34, 36, 38])/38

# dsgdb9nsd_003886
# I1_large = np.array([0, 4, 6, 8])/38
# I2_large = np.array([8, 12, 14, 15])/38
# I3_large = np.array([15, 16, 18, 20, 22, 23])/38
# I4_large = np.array([23, 24, 26, 28, 30])/38
# I5_large = np.array([30, 32, 34, 36, 38])/38

intervals_large = [I1_large, I2_large, I3_large, I4_large, I5_large]
intervals_idc_large = []

for i in intervals_large:
    intervals_idc_large.extend(np.where(np.isin(lam_vals, i)))

In [ ]:
# integration with trapezoidal rule
ae_large_MIC_trapz, ae_large_trapz = atomic_energies_piecewise_integration(intervals_idc_large, 'trapz', lam_vals, densities, nuclei, gpts, h_matrix)

In [ ]:
# integration with cubic splines
ae_large_MIC_cs, ae_large_cs = atomic_energies_piecewise_integration(intervals_idc_large, 'cubic-splines', lam_vals, densities, nuclei, gpts, h_matrix)

In [ ]:
interval_string = '[0.0, 4.0]/38, [4.0, 6.0]/38, [6.0, 8.0]/38, [8.0, 10.0]/38, [10.0, 12.0]/38, [12.0, 14.0]/38, [14.0, 15.0]/38, [15.0, 16.0]/38, [16.0, 18.0]/38, [18.0, 20.0]/38, [20.0, 22.0]/38, [22.0, 23.0]/38, [23.0, 24.0]/38, [24.0, 26.0]/38, [26.0, 28.0]/38, [28.0, 29.0]/38, [29.0, 30.0]/38, [30.0, 32.0]/38, [32.0, 34.0]/38, [34.0, 36.0]/38, [36.0, 38.0]/38'
# trapezoidal
header_trapz = 'atomic energies for dsgdb9nsd_001212 for piecewise integration over lambda with the trapezoidal rule\n'
header_trapz += 'every line corresponds to an integration interval, the intervals are ' + interval_string + '\n'
header_trapz += 'every column corresponds to an atom where the sum of values in a column is the atomic energy of the atom'
path = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/integration/'
path_ae_trapz = path + 'ae_trapz.txt'
np.savetxt(path_ae_trapz, ae_large_trapz, delimiter='\t', header=header_trapz)
# trapezoidal mic
header_trapz_mic = 'atomic energies for dsgdb9nsd_001212 for piecewise integration over lambda with the trapezoidal rule and the minimum image convention for integration over r\n'
header_trapz_mic += 'every line corresponds to an integration interval, the intervals are ' + interval_string + '\n'
header_trapz_mic += 'every column corresponds to an atom where the sum of values in a column is the atomic energy of the atom'
path = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/integration/'
path_ae_trapz_mic = path + 'ae_MIC_trapz.txt'
np.savetxt(path_ae_trapz_mic, ae_large_MIC_trapz, delimiter='\t', header=header_trapz_mic)

# cubic splines
header_cs = 'atomic energies for dsgdb9nsd_001212 for piecewise integration over lambda using cubic splines\n'
header_cs += 'every line corresponds to an integration interval, the intervals are ' + interval_string + '\n'
header_cs += 'every column corresponds to an atom where the sum of values in a column is the atomic energy of the atom'
path = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/integration/'
path_ae_cs = path + 'ae_cs.txt'
np.savetxt(path_ae_cs, ae_large_cs, delimiter='\t', header=header_cs)
# cubic splines mic
header_cs_mic = 'atomic energies for dsgdb9nsd_001212 for piecewise integration over lambda using cubic splines and the minimum image convention for integration over r\n'
header_cs_mic += 'every line corresponds to an integration interval, the intervals are ' + interval_string + '\n'
header_cs_mic += 'every column corresponds to an atom where the sum of values in a column is the atomic energy of the atom'
path = '/home/misa/APDFT/prototyping/atomic_energies/results/test_calculations/integration/'
path_ae_cs_mic = path + 'ae_MIC_cs.txt'
np.savetxt(path_ae_cs_mic, ae_large_MIC_cs, delimiter='\t', header=header_cs_mic)

# Plot Results

## Large grid compare trapezoidal and cubic splines without MIC

In [ ]:
diff_trapz_cs = ae_large_trapz.sum(axis=1)-ae_large_cs.sum(axis=1)
lam_width = np.zeros(len(diff_trapz_cs))
for idx in range(1, len(lam_vals)):
    lam_width[idx-1] = lam_vals[idx]-lam_vals[idx-1]

diff_trapz_cs_norm = diff_trapz_cs/lam_width

intervals = lam_vals[0:len(lam_vals)-1]+lam_width/2

fig, ax = plt.subplots(1,2)
ax[0].bar(intervals, diff_trapz_cs, width = 0.02)
ax[0].set_xlabel('Midpoint of integration interval')
ax[0].set_ylabel(r'$\int_{trapz} - \int_{cubic splines}$')

ax[1].bar(intervals, diff_trapz_cs_norm, width = 0.02)
ax[1].set_xlabel(r'Midpoint of integration interval $[a, b]$')
ax[1].set_ylabel(r'$\frac{\int_{trapz} - \int_{cubic splines}}{b-a}$')

# Testing

## Selection of indices for integration over subset of all lambda points

In [ ]:
# return indices of lambda values and densities over which integration is performed
lam_vals_test = np.array([0,4,6,8,10,12,14,15,16,18,20,22,23,24,26,28,29,30,32,34,36,38])/38

I1 = [0, 8/38]
I2 = [8/38, 15/38]
I3 = [15/38, 23/38]
I4 = [23/38, 30/38]
I5 = [30/38, 38/38]

intervals_test = [I1, I2, I3, I4, I5]
intervals_idc_test = []

for i in intervals_test:
    intervals_idc_test.extend(np.where(np.isin(lam_vals_test, i)))
intervals_idc_test

In [ ]:
# get all lambda values for generation of splines
unique_idc_test = []
for el in intervals_idc_test:
    unique_idc_test.extend(el.tolist())

unique_idc_test = list(set(unique_idc_test))
unique_idc_test.sort()
unique_idc_test

## Calculation of atomic energies
Reference values are from old code in number_lambda_values.ipyn

### Trapezoidal small grid

In [ ]:
# define intervals, lambda-values
I1 = [0, 8/38]
I2 = [8/38, 15/38]
I3 = [15/38, 23/38]
I4 = [23/38, 30/38]
I5 = [30/38, 38/38]

intervals_small = [I1, I2, I3, I4, I5]
intervals_idc_small = []

for i in intervals_small:
    intervals_idc_small.extend(np.where(np.isin(lam_vals, i)))

In [ ]:
# integration with trapezoidal rule and small grid
ae_small_MIC_trapz, ae_small_trapz = atomic_energies_piecewise_integration(intervals_idc_small, 'trapz', lam_vals, densities, nuclei, gpts, h_matrix)

In [ ]:
ae_small_trapz_ref = np.array([[ -3.8629316475752997,  -4.157522608670569 ,  -5.377488652563657 ,
         -5.619910748456998 ,  -4.057659441008973 ,  -3.885545113870597 ,
         -4.30672732286963  ,  -0.6099499329868788,  -0.6177173743838867,
         -0.6174587920281657,  -0.643318577517304 ,  -0.6431330437870848],
       [ -4.9359858626763975,  -5.69396226849638  ,  -7.186950401690477 ,
         -7.789775969122621 ,  -5.435883972188529 ,  -5.039734420597661 ,
         -5.362295322614413 ,  -0.7338511120718167,  -0.7510048640949457,
         -0.7508045385081719,  -0.8151785504621742,  -0.8150956854261442],
       [ -7.639793657676979 ,  -9.106296812113062 , -11.535505382511882 ,
        -12.64367330638071  ,  -8.578940593567179 ,  -7.896107159675002 ,
         -8.316017736415942 ,  -1.0907433842335372,  -1.1203196438563396,
         -1.1200719415172806,  -1.2359760094587156,  -1.2357271303538035],
       [ -8.613867137001069 , -10.386399214408923 , -13.554895095780992 ,
        -14.833770413015134 ,  -9.71201046849548  ,  -8.953466878621022 ,
         -9.54032807159007  ,  -1.195823954667783 ,  -1.227588858932992 ,
         -1.2271935784343124,  -1.3544784050865106,  -1.3544474137182823],
       [-12.105959920539576 , -14.617421489376792 , -20.131305116389775 ,
        -21.952503577172198 , -13.663789364342634 , -12.602283795999682 ,
        -13.778136651270598 ,  -1.618593600803003 ,  -1.6616069120272778,
         -1.6608080220694932,  -1.850761065414696 ,  -1.851367101475221 ]])

In [ ]:
np.allclose(ae_small_trapz,ae_small_trapz)

### Trapezoidal large grid

In [ ]:
# integration with trapezoidal rule and large grid
I1_large = np.array([0, 4, 6, 8])/38
I2_large = np.array([8, 10, 12, 14, 15])/38
I3_large = np.array([15, 16, 18, 20, 22, 23])/38
I4_large = np.array([23, 24, 26, 28, 29, 30])/38
I5_large = np.array([30, 32, 34, 36, 38])/38

intervals_large = [I1_large, I2_large, I3_large, I4_large, I5_large]
intervals_idc_large = []

for i in intervals_large:
    intervals_idc_large.extend(np.where(np.isin(lam_vals, i)))


In [ ]:
# integration with trapezoidal rule
ae_large_MIC_trapz, ae_large_trapz = atomic_energies_piecewise_integration(intervals_idc_large, 'trapz', lam_vals, densities, nuclei, gpts, h_matrix)

In [ ]:
ae_large_trapz

In [ ]:
ae_large_trapz_ref = np.array([[-1.692523364286604  , -1.7615802245882732 , -2.2983912597614258 ,
        -2.3639127777005235 , -1.7431992244505499 , -1.6928973952596964 ,
        -1.903003808226167  , -0.27401446558355425, -0.2765975268775773 ,
        -0.27646542401243657, -0.28382408995037417, -0.28372426239046583],
       [-1.0106911479508383 , -1.0955332663635242 , -1.4037125324066966 ,
        -1.4829487014834004 , -1.068887856174829  , -1.013476736297696  ,
        -1.1103694389039112 , -0.15909937940885072, -0.16127717786861734,
        -0.16122652067757418, -0.16901702261879606, -0.16897169949050206],
       [-1.1301623777013632 , -1.2541238062370315 , -1.5988890656668984 ,
        -1.705969999747387  , -1.2117031042017954 , -1.1384143171354961 ,
        -1.2355493655082341 , -0.17457962986379352, -0.17740775802580014,
        -0.1773585066783975 , -0.18793462202293515, -0.18789282924204034],
       [-1.2498903457690305 , -1.4144567645346495 , -1.7941033099195707 ,
        -1.929033331973013  , -1.3593346425031334 , -1.2660021110885742 ,
        -1.3621702810564267 , -0.18979625658595742, -0.19335205497870214,
        -0.19330092116512132, -0.20796168971598125, -0.2079338239722323 ],
       [-1.3709285345049123 , -1.5762433578569597 , -1.9886356526250564 ,
        -2.157568711998892  , -1.5104990251651884 , -1.3981509109082206 ,
        -1.4916702651563414 , -0.20519315586557085, -0.2092874162660573 ,
        -0.20922338129908666, -0.2279747384687971 , -0.22797707233612274],
       [-1.4970447012098362 , -1.7440810322628701 , -2.1933439319059396 ,
        -2.3928044191441007 , -1.6643314105133309 , -1.5360891543683506 ,
        -1.6261375698015494 , -0.220765358605131  , -0.22601109071360395,
        -0.22594013370721663, -0.2478377004132676 , -0.2478419691031576 ],
       [-0.7982386700022888 , -0.9373443237242045 , -1.1767918647800597 ,
        -1.2865752188136654 , -0.8911030585690956 , -0.821011598219087  ,
        -0.8649951658030182 , -0.11647324801284852, -0.11959814861997317,
        -0.11956462823070885, -0.13149734846661654, -0.13149270108204392],
       [-0.8327848521782706 , -0.9816524287602421 , -1.2316247602954529 ,
        -1.3485551228673252 , -0.9311660408206368 , -0.856991030126512  ,
        -0.9005986971528597 , -0.12101083219194224, -0.12410175494388996,
        -0.12406942065812412, -0.13656686811277863, -0.13656279267708954],
       [-1.7690406885121825 , -2.0945828894810656 , -2.6315226208290468 ,
        -2.886376899921966  , -1.9816683532895687 , -1.8207287213928873 ,
        -1.90855287488083   , -0.256046082725375  , -0.261518461149154  ,
        -0.26146314416844146, -0.28803859061303344, -0.28803838108646557],
       [-1.9062919881364464 , -2.2719367657117164 , -2.8645420669654884 ,
        -3.145540461674422  , -2.1409859630000905 , -1.9657533382286911 ,
        -2.0592563818447527 , -0.27374667146331744, -0.27913879299843947,
        -0.279072089476143  , -0.3080953962239717 , -0.3081115586496456 ],
       [-2.0448963499428743 , -2.4525133679423026 , -3.1069167662291024 ,
        -3.4128835366909622 , -2.302740331187998  , -2.1163527734391043 ,
        -2.222228643598547  , -0.2908735397090068 , -0.2968673780389877 ,
        -0.29680834729773525, -0.3288483254505954 , -0.3288025500853118 ],
       [-1.076011880190656  , -1.294214418609714  , -1.6472620725648575 ,
        -1.8087640278995065 , -1.2129128941139935 , -1.1156864431345082 ,
        -1.1746660882164344 , -0.15191856632278114, -0.15556172363300272,
        -0.15553927302655257, -0.17232589298938933, -0.17226778487820069],
       [-1.1133752340660452 , -1.3397703914756645 , -1.7129700430577355 ,
        -1.879214525664933  , -1.2545869132687173 , -1.1545081784370526 ,
        -1.2183125150324916 , -0.1562367617796211 , -0.16090045924485738,
        -0.16085406015975126, -0.17756590351547735, -0.1775486964179631 ],
       [-2.3387315001209115 , -2.8179508421326775 , -3.6302439944666385 ,
        -3.9761334122361185 , -2.634483092370245  , -2.4273345108620092 ,
        -2.5711029158115046 , -0.32590137784480117, -0.3366783709572932 ,
        -0.3365486643476187 , -0.37046479071049787, -0.370521483604585  ],
       [-2.491508856645569  , -3.0064962324265183 , -3.9193044724201402 ,
        -4.283206486928603  , -2.8066992285194723 , -2.588922584408965  ,
        -2.757136973647851  , -0.3446895424952698 , -0.3559445755930297 ,
        -0.35580267897973   , -0.39118940829437643, -0.3912600393681325 ],
       [-1.305355147216178  , -1.5757980120384634 , -2.0741888934960206 ,
        -2.2641955157631517 , -1.470792212736105  , -1.3571935346342854 ,
        -1.4517101730006134 , -0.17988440209904236, -0.18526879508466007,
        -0.1851901513651735 , -0.2036279785643613 , -0.20366783553613704],
       [-1.3461436830308067 , -1.625526241759999  , -2.153102196573092  ,
        -2.350554511754124  , -1.5175272983206887 , -1.4009580163696036 ,
        -1.5032846392770927 , -0.18508555151815503, -0.19010507051236059,
        -0.1900224522371243 , -0.2091966812693022 , -0.2092423489104389 ],
       [-2.8124265256801677 , -3.3960255261921057 , -4.543816472095645  ,
        -4.966760964438669  , -3.174743249078513  , -2.9324064278342963 ,
        -3.1635827196060307 , -0.38475969096685275, -0.39396144875953276,
        -0.39378744658283044, -0.43500831323396283, -0.43512209350862074],
       [-2.971353004428405  , -3.586788255449936  , -4.869816399922449  ,
        -5.333082574036901  , -3.3604712781543826 , -3.1028346516855416 ,
        -3.3745874324799296 , -0.40253942731149833, -0.4120628680240457 ,
        -0.41142708748222645, -0.45863382574793227, -0.4583528720016104 ],
       [-3.132961391393489  , -3.782538807517113  , -5.223791222021115  ,
        -5.714471603139713  , -3.5454160297661677 , -3.2706546656488884 ,
        -3.587995871911982  , -0.4190246256838246 , -0.429823959859265  ,
        -0.4291695816603587 , -0.4821613805534805 , -0.48191175002447095],
       [-3.2674594264655807 , -3.944954020348132  , -5.544529383258778  ,
        -6.047140629614974  , -3.6932365752213627 , -3.404577220791423  ,
        -3.7624810253608607 , -0.43068722455441305, -0.44211135644149474,
        -0.4418872783268709 , -0.4971482765254228 , -0.4973451984058816 ]])

In [ ]:
np.array_equal(ae_large_trapz, ae_large_trapz_ref)

### Cubic splines small grid

In [ ]:
# define intervals, lambda-values
I1 = [0, 8/38]
I2 = [8/38, 15/38]
I3 = [15/38, 23/38]
I4 = [23/38, 30/38]
I5 = [30/38, 38/38]

intervals_small = [I1, I2, I3, I4, I5]
intervals_idc_small = []

for i in intervals_small:
    intervals_idc_small.extend(np.where(np.isin(lam_vals, i)))

In [ ]:
# integration with cs and small grid
ae_small_MIC_cs_test, ae_small_cs_test = atomic_energies_piecewise_integration(intervals_idc_small, 'cubic-splines', lam_vals, densities, nuclei, gpts, h_matrix)

In [ ]:
ae_small_cs_ref = np.array([[ -3.6681304794399407,  -3.8995402176559413,  -5.068370388262649 ,
         -5.260611739809258 ,  -3.820578578687523 ,  -3.6805482546951107,
         -4.109011853928227 ,  -0.5848301745724862,  -0.5912360140526087,
         -0.590975325231234 ,  -0.6114310285154211,  -0.6112258578616918],
       [ -4.958369553616252 ,  -5.725132094754784 ,  -7.21560876855823  ,
         -7.829167109410749 ,  -5.465965402806441 ,  -5.05983167758914  ,
         -5.375451531928247 ,  -0.7372753269189571,  -0.7547832821421987,
         -0.7545843745523084,  -0.8201454389150129,  -0.8200967396445614],
       [ -7.603238959730746 ,  -9.071092558960336 , -11.446711552179293 ,
        -12.560632834318971 ,  -8.547387274959323 ,  -7.861226523180001 ,
         -8.259508903056762 ,  -1.0856747731801688,  -1.1156284508386962,
         -1.1153993808921723,  -1.2329726696743446,  -1.232681057365211 ],
       [ -8.609826760323315 , -10.387502225537093 , -13.512314448225444 ,
        -14.792184437752514 ,  -9.708167123948533 ,  -8.949346500087193 ,
         -9.520025914757799 ,  -1.197000207088908 ,  -1.2287110845308389,
         -1.228340171007377 ,  -1.3550723462225513,  -1.3549731288108005],
       [-12.135920210772149 , -14.654968642497227 , -20.10840961391662  ,
        -21.91536106515243  , -13.694214879299803 , -12.64279822281538  ,
        -13.800656941352104 ,  -1.6292848730878384,  -1.6722210719781219,
         -1.6713432450138743,  -1.8551573256207388,  -1.855961274193183 ]])

In [ ]:
np.array_equal(ae_small_cs_test, ae_small_cs_ref)

### Cubic splines large grid

In [ ]:
# integration with trapezoidal rule and large grid
I1_large = np.array([0, 4, 6, 8])/38
I2_large = np.array([8, 10, 12, 14, 15])/38
I3_large = np.array([15, 16, 18, 20, 22, 23])/38
I4_large = np.array([23, 24, 26, 28, 29, 30])/38
I5_large = np.array([30, 32, 34, 36, 38])/38

intervals_large = [I1_large, I2_large, I3_large, I4_large, I5_large]
intervals_idc_large = []

for i in intervals_large:
    intervals_idc_large.extend(np.where(np.isin(lam_vals, i)))

In [ ]:
ae_large_MIC_cs_test, ae_large_cs_test = atomic_energies_piecewise_integration(intervals_idc_large, 'cubic-splines', lam_vals, densities, nuclei, gpts, h_matrix)

In [ ]:
np.set_printoptions(precision=20)
ae_large_cs_test

In [ ]:
ae_large_cs_ref = np.array([[-1.648062295605488  , -1.7037762505769423 , -2.2295824846321173 ,
        -2.2817581159654927 , -1.6903426168461668 , -1.6478119690457669 ,
        -1.8592163500380872 , -0.26800531188434173, -0.2704540134024333 ,
        -0.2703190930705093 , -0.27674068301348403, -0.2766392625193489 ],
       [-1.0121649984314733 , -1.0971460240446154 , -1.4049229426688434 ,
        -1.4853959167786706 , -1.0706655325944823 , -1.0146171752936426 ,
        -1.111049332695472  , -0.15935970076367292, -0.16151610665859356,
        -0.16146619060940196, -0.1692945163158796 , -0.16924944374918358],
       [-1.1297491134446085 , -1.2534707089118182 , -1.5984619104200242 ,
        -1.7054552667415797 , -1.2106920094966336 , -1.137871471057753  ,
        -1.2352411205304055 , -0.17455162841102836, -0.17736309421004384,
        -0.1773136604317376 , -0.18771346424078347, -0.18767104287953898],
       [-1.249941183156844  , -1.414585186942354  , -1.7945826651486871 ,
        -1.9286240317201662 , -1.359193199766953  , -1.2657079498459605 ,
        -1.3620028727042475 , -0.18977975290858445, -0.1933849230538903 ,
        -0.1933354219769813 , -0.20801409300305068, -0.20798329267422883],
       [-1.3704833388763094 , -1.5756559480390098 , -1.9874863377290681 ,
        -2.15698462211597   , -1.5103094686347112 , -1.3975898990231326 ,
        -1.4912086092670291 , -0.20519703574910356, -0.20920507561383306,
        -0.20914020147468593, -0.22799329257278655, -0.22799985707378928],
       [-1.4962356573519178 , -1.7432767085851633 , -2.1923738679730778 ,
        -2.392199915368953  , -1.6638970780283797 , -1.5357326413310752 ,
        -1.6256679414383326 , -0.22067869101840037, -0.22591342866721587,
        -0.22584120605587718, -0.2477861241144138 , -0.2477905372489238 ],
       [-0.7981617500380442 , -0.9373262964258592 , -1.1767339249822284 ,
        -1.2864486342910628 , -0.8910568603083602 , -0.8209703314312147 ,
        -0.8649325379897954 , -0.11643464455806891, -0.11959350281028042,
        -0.11956012061981226, -0.13149745841202398, -0.1314925066788247 ],
       [-0.8327834854791698 , -0.9817204790863061 , -1.231586455077101  ,
        -1.348504209036317  , -0.9311848939058701 , -0.8570358228125385 ,
        -0.9006209330510422 , -0.12098894723126719, -0.12410630285738039,
        -0.12407361107019123, -0.13657426065329104, -0.13657042462491925],
       [-1.7692095274647377 , -2.0943828153478803 , -2.630480641224574  ,
        -2.885695040907031  , -1.9817766568871797 , -1.820593200890658  ,
        -1.9079318999949435 , -0.2562018938892328 , -0.26153443960972544,
        -0.2614827126287359 , -0.2880481262772628 , -0.28804388800398034],
       [-1.9062495217525348 , -2.271587511006466  , -2.8639058965088373 ,
        -3.144852398690777  , -2.140719335998144  , -1.965146709567712  ,
        -2.0580223587811894 , -0.27378460266586735, -0.279183572387736  ,
        -0.27911358157790134, -0.3080100637978025 , -0.30803575712686443],
       [-2.044220845434863  , -2.4524990978965273 , -3.1058258010965925 ,
        -3.4122278910084676 , -2.3024801563594974 , -2.1161273993979313 ,
        -2.2216513333437726 , -0.2908528639810761 , -0.2966181554255984 ,
        -0.2965591541734596 , -0.3288320576831964 , -0.3287913423226795 ],
       [-1.0759072797155693 , -1.294198525801865  , -1.6470695545796492 ,
        -1.8086157097913789 , -1.2128654635034286 , -1.1156619177582041 ,
        -1.1745937918637201 , -0.15192027700359614, -0.15552506104132133,
        -0.15550542578454854, -0.17232710317425046, -0.17226191460500803],
       [-1.1133971019605802 , -1.3397490955942857 , -1.7128982341750847 ,
        -1.879140324548957  , -1.2545982280537884 , -1.1544782417853139 ,
        -1.218252459691627  , -0.15622633103711298, -0.1609308109468187 ,
        -0.16088405341077625, -0.17757563129724735, -0.17755692602748935],
       [-2.3385683421114156 , -2.8175281756319945 , -3.6283899515459965 ,
        -3.9744689479683135 , -2.6341076872393634 , -2.426991789726477  ,
        -2.5704783205248476 , -0.3258338276495024 , -0.33680636563388056,
        -0.33666701334273735, -0.3704640508669623 , -0.370548857117853  ],
       [-2.490643720191077  , -3.005938304373858  , -3.917666638626231  ,
        -4.280970561204246  , -2.805779651149579  , -2.5881824584581974 ,
        -2.7560260670232175 , -0.34447137418711543, -0.3558885874434345 ,
        -0.3557490249020012 , -0.3911114233974508 , -0.39117460590178205],
       [-1.3053116428916853 , -1.575688313279326  , -2.0740353274109653 ,
        -2.2639086546912237 , -1.470676070530133  , -1.3570618438603068 ,
        -1.4515265194096443 , -0.17988689042390055, -0.18527134603041992,
        -0.18519336007908732, -0.20360783296998317, -0.2036485973010562 ],
       [-1.3461863163820231 , -1.6255818711726382 , -2.1530567381765806 ,
        -2.3504044766772116 , -1.5175108065719778 , -1.4009435762875482 ,
        -1.503214713425702  , -0.18511065369963298, -0.1901204401670255 ,
        -0.1900344904089751 , -0.20920130116874888, -0.20924355132375794],
       [-2.8128784951556467 , -3.3969706193944837 , -4.5438461277556215 ,
        -4.965994568002953  , -3.175147596451267  , -2.9331062960413754 ,
        -3.16372716127495   , -0.384949224393441  , -0.3940370263950086 ,
        -0.3939293408643589 , -0.43489943878580656, -0.435076636094837  ],
       [-2.970528339579866  , -3.585464091036091  , -4.866018196133864  ,
        -5.330619889224213  , -3.3598501017199327 , -3.1024657269321603 ,
        -3.3736449316430512 , -0.4025921939154672 , -0.41198922419347433,
        -0.41130879907611045, -0.45849103153018955, -0.4581667012465865 ],
       [-3.134467127203742  , -3.784396460565395  , -5.2249669343795535 ,
        -5.717555029073855  , -3.5478880268632587 , -3.272734601399271  ,
        -3.5904195703114934 , -0.4192712948275201 , -0.430213121260721  ,
        -0.42944941436392375, -0.4828904107268154 , -0.4825378406934131 ],
       [-3.2758163240195852 , -3.9555152421869195 , -5.560243068044147  ,
        -6.061629612004886  , -3.702531866345772  , -3.413575131953709  ,
        -3.7731903755912644 , -0.43205908112653296, -0.4433652022927366 ,
        -0.4433986346745371 , -0.49864527946662696, -0.4990897213058603 ]])

In [ ]:
np.array_equal(ae_large_cs_test, ae_large_cs_ref)

In [ ]:
ae_large_cs_test.sum()

In [ ]:
ae_large_cs_ref.sum()